In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [40]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from plug_mongo import get_entries
df = pd.DataFrame(get_entries()).drop(["_id", "timestamp"], axis=1).set_index("game_id")
df.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_8_w,player_8_l,player_8_co_w,player_8_co_l,player_8_ct_wr,player_9_w,player_9_l,player_9_co_w,player_9_co_l,player_9_ct_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,216,181,145,91,0.5354,174,147,8,3,...,297,284,1,0,0.5003,941,930,43,35,0.5093
4484824702,0,541,503,27,28,0.5161,160,111,0,0,...,269,202,9,3,0.4971,988,954,65,78,0.4948
548810655,1,155,146,20,6,0.4879,55,46,0,0,...,186,144,22,23,0.5088,55,45,0,1,0.5229
6140028214,1,686,645,43,41,0.5082,368,281,17,11,...,234,198,25,32,0.5197,2090,2035,390,340,0.5306
1351099000,1,31,14,23,8,0.5043,38,12,2,0,...,17,6,8,3,0.5148,125,103,0,0,0.4933


In [7]:
def add_one(column):
	return df[column].apply(lambda x: x+1)

def get_winrate(column):
	win = add_one(column+'_w')
	los = add_one(column+'_l')
	return (win / (win + los)).rename(column+"_wr")

get_winrate('player_0').head()

game_id
548809662     0.543860
4484824702    0.518164
548810655     0.514851
6140028214    0.515379
1351099000    0.680851
Name: player_0_wr, dtype: float64

In [9]:
result = [df]
result.extend([get_winrate(f"player_{x}") for x in range(10)])
result.extend([get_winrate(f"player_{x}_co") for x in range(10)])
df_winrates = pd.concat(result, axis=1)
df_winrates.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,216,181,145,91,0.5354,174,147,8,3,...,0.613445,0.692308,0.789474,0.482759,0.700000,0.500000,0.584211,0.666667,0.666667,0.550000
4484824702,0,541,503,27,28,0.5161,160,111,0,0,...,0.491228,0.500000,0.533537,0.727273,0.545763,0.428571,0.666667,0.549020,0.714286,0.455172
548810655,1,155,146,20,6,0.4879,55,46,0,0,...,0.750000,0.500000,0.500000,0.285714,0.333333,0.500000,0.500000,0.500000,0.489362,0.333333
6140028214,1,686,645,43,41,0.5082,368,281,17,11,...,0.511628,0.600000,0.764706,0.625000,0.800000,0.642857,0.527538,0.606383,0.440678,0.534153
1351099000,1,31,14,23,8,0.5043,38,12,2,0,...,0.727273,0.750000,0.428571,0.500000,0.666667,0.531469,0.666667,0.500000,0.692308,0.500000


In [10]:
train_dataset = df_winrates.sample(frac=0.8, random_state=0)
test_dataset = df_winrates.drop(train_dataset.index)
train_dataset.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
3252704800,0,419,427,29,15,0.5127,32,24,2,0,...,0.652174,0.750000,0.666667,0.651163,0.750000,0.647059,0.594203,0.655738,0.500000,0.500000
4488699054,0,359,316,331,280,0.5298,134,101,2,1,...,0.541599,0.600000,0.833333,0.444444,0.500000,0.800000,0.513514,0.521739,0.560606,0.454545
3254899506,0,201,221,6,8,0.4836,266,226,1,1,...,0.437500,0.500000,0.571429,0.636364,0.750000,0.571429,0.583333,0.697674,0.333333,0.500000
6145685471,0,366,316,19,14,0.4927,554,514,8,15,...,0.571429,0.360000,0.618182,0.600000,0.666667,0.488372,0.429630,0.629213,0.600000,0.555556
1237727628,0,104,85,28,14,0.5012,152,154,3,4,...,0.659091,0.444444,0.600000,0.421053,0.567901,0.857143,0.600000,0.333333,0.625000,0.384615


In [11]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')
train_labels.head()

game_id
3252704800    0
4488699054    0
3254899506    0
6145685471    0
1237727628    0
Name: label, dtype: int64

In [12]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_features)

In [14]:
def build_and_compile_model(norm, width, depth, activation='relu'):
    layer_l = [norm]
    layer_l.extend([layers.Dense(width, activation=activation) for _ in range(depth)])
    layer_l.append(layers.Dense(1, activation='softmax'))
    model = keras.Sequential(layer_l)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [15]:
dnn_model = build_and_compile_model(normalizer, 64, 2)
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 70)               141       
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                4544      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,910
Trainable params: 8,769
Non-trainable params: 141
_________________________________________________________________


In [16]:
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
367/367 [==============================] - 3s 5ms/step - loss: 0.5125 - val_loss: 0.4987
Epoch 2/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4705 - val_loss: 0.4948
Epoch 3/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4533 - val_loss: 0.4957
Epoch 4/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4413 - val_loss: 0.4963
Epoch 5/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4307 - val_loss: 0.4935
Epoch 6/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4202 - val_loss: 0.5036
Epoch 7/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4116 - val_loss: 0.5006
Epoch 8/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4036 - val_loss: 0.5038
Epoch 9/100
367/367 [==============================] - 2s 4ms/step - loss: 0.3952 - val_loss: 0.5050
Epoch 10/100
367/367 [==============================] - 2s 5ms/step - loss: 0.3883 - val_lo

In [48]:
early_stop = keras.callbacks.EarlyStopping(patience=5, verbose=0, restore_best_weights=True)
activations = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']

def test_model(width, depth, activation, verbose=1):
    with tf.device("GPU:0"):
        model = build_and_compile_model(normalizer, width, depth, activation)
        history_return = model.fit(
            train_features,
            train_labels,
            validation_split=0.2,
            verbose=verbose, epochs=100,
            callbacks = [early_stop]
        )
    return min(history_return.history['val_loss']), len(history_return.history['val_loss'])

In [34]:
[(act, test_model(64, 2, act, 0)) for act in activations]

Epoch 1/100
367/367 [==============================] - 2s 5ms/step - loss: 0.5056 - val_loss: 0.4953
Epoch 2/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4716 - val_loss: 0.4845
Epoch 3/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4546 - val_loss: 0.4914
Epoch 4/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4439 - val_loss: 0.4936
Epoch 5/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4326 - val_loss: 0.4941
Epoch 6/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4229 - val_loss: 0.4920
Epoch 7/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4143 - val_loss: 0.4927
Epoch 7: early stopping
Epoch 1/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4866 - val_loss: 0.4851
Epoch 2/100
367/367 [==============================] - 2s 4ms/step - loss: 0.4773 - val_loss: 0.4812
Epoch 3/100
367/367 [==============================] - 2s 4ms/step 

[('relu', 0.4844696521759033),
 ('sigmoid', 0.4628167152404785),
 ('softmax', 0.4676401615142822),
 ('softplus', 0.47889775037765503),
 ('softsign', 0.4780551791191101),
 ('tanh', 0.4837893843650818),
 ('selu', 0.48302292823791504),
 ('elu', 0.48381277918815613),
 ('exponential', nan)]

In [53]:
modes = []
for x in [18, 72, 144]:
    for y in [1, 2, 4]:
        for z in ['softmax']:
            modes.append((x, y, z))
results = [(mode, test_model(mode[0], mode[1], mode[2], 0)) for mode in modes]
results

[((18, 1, 'softmax'), (0.4686773419380188, 27)),
 ((18, 2, 'softmax'), (0.46939608454704285, 13)),
 ((18, 4, 'softmax'), (0.4693832993507385, 15)),
 ((72, 1, 'softmax'), (0.4719468653202057, 11)),
 ((72, 2, 'softmax'), (0.4623214304447174, 16)),
 ((72, 4, 'softmax'), (0.48021045327186584, 12)),
 ((144, 1, 'softmax'), (0.46452662348747253, 13)),
 ((144, 2, 'softmax'), (0.46166375279426575, 18)),
 ((144, 4, 'softmax'), (0.48021358251571655, 16))]

In [54]:
sorted(results, key=lambda x: x[1][0])

[((144, 2, 'softmax'), (0.46166375279426575, 18)),
 ((72, 2, 'softmax'), (0.4623214304447174, 16)),
 ((144, 1, 'softmax'), (0.46452662348747253, 13)),
 ((18, 1, 'softmax'), (0.4686773419380188, 27)),
 ((18, 4, 'softmax'), (0.4693832993507385, 15)),
 ((18, 2, 'softmax'), (0.46939608454704285, 13)),
 ((72, 1, 'softmax'), (0.4719468653202057, 11)),
 ((72, 4, 'softmax'), (0.48021045327186584, 12)),
 ((144, 4, 'softmax'), (0.48021358251571655, 16))]

In [55]:
test_model(288, 2, 'softmax')

Epoch 1/100
367/367 [==============================] - 3s 6ms/step - loss: 0.5172 - val_loss: 0.5144
Epoch 2/100
367/367 [==============================] - 2s 5ms/step - loss: 0.5116 - val_loss: 0.5079
Epoch 3/100
367/367 [==============================] - 2s 5ms/step - loss: 0.5044 - val_loss: 0.5011
Epoch 4/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4942 - val_loss: 0.4913
Epoch 5/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4775 - val_loss: 0.4769
Epoch 6/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4585 - val_loss: 0.4724
Epoch 7/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4395 - val_loss: 0.4677
Epoch 8/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4149 - val_loss: 0.4666
Epoch 9/100
367/367 [==============================] - 2s 5ms/step - loss: 0.4014 - val_loss: 0.4666
Epoch 10/100
367/367 [==============================] - 2s 5ms/step - loss: 0.3920 - val_lo

(0.4666041433811188, 14)